In [1]:
from sympy import *
from mpmath import *
from numpy import *
from sympy import integrate, log, exp, cos, oo, sin
import math
from scipy.special import *
from sympy import symbols, diff
from numpy.polynomial import legendre as leg

                                         Задаём функцию правой части

In [2]:
def f(x):
    return sin(x)

                                                Задаём ядро

In [3]:
def K(x,t):
    return 1-x*cos(x*t)

                                               Разложение ядра

In [4]:
def Hn(x, s, n):
    R = []
    for k in range(0,n+1):
        res  = ((1/factorial(k, exact=False))*(diff(K(x,s), x ,k)).subs(x,'0'))
        R.append(res)
    #print(R)

    B = []
    for k in range(0,n+1):
        res  = R[k]*(x**k)
        B.append(res)
    return B

                         Вычисление приближенного решения заменой ядра на вырожденное

In [5]:
def Zamena(x, s, a, b, n):

                    # Генерируем список искомых коэффициентов для уравнения
        
    ci = [Symbol("ci_%i" % k) for k in range(n+1)]

                        # Вычисляем разложение на матрицы Ai(x) и Bi(s)
    R = []
    for k in range(0,n+1):
        res  = ((1/factorial(k, exact=False))*(diff(K(x,s), x ,k)).subs(x,'0'))
        R.append(res)

    B = []
    for k in range(0,n+1):
        res  = R[k]*(x**k)
        B.append(res.subs(s,'1'))
    
    N = []
    for k in range(0,n+1):
        r  = B[k]
        N.append(r.subs(x,'1'))
    
    M = []
    for k in range(0,n+1):
        if B[k] == 0:
            ir = N[k]
        else:
            ir  = B[k]*N[k]**(-1)
        M.append(ir)
    
################################
    An = []
    for k in range(0,n+1):
        h  = M[k]*ci[k]
        An.append(h)
    
################################

                                            #решение ищем в виде
    
    itog = f(x) + sum((M[k]*ci[k] for k in range(0,n+1)))

    
                                    # Вычисляем коэффициенты правой части
        
    eq_2 = [integrate(f(s)*R[k], (s, a, b)) \
            for k in range(0,n+1)]
    
                                            # Вычисляем левую часть
        
    V = []
    for j in range(0,n+1):
        eq_3 = sum(integrate(M[k].subs(x,s)*R[j]*ci[k], (s, a, b)) \
                for k in range(0,n+1))
        V.append(eq_3)

    it = [An[k].subs(x,'1') for k in range(0,n+1)]
    
                                    # Генерируем систему уравнений
        
    System = []
    for k in range(0,n+1):
        if (it[k] == 0): 
            continue
        elif (V[k] == 0):
            continue
        else:
            system = Eq(it[k] - V[k],eq_2[k])
            System.append(system)

                            # Решаем систему и находим коэффициенты

    coeffs = solve(System, ci)
    #print(coeffs)
    
    return itog.subs(coeffs)

In [6]:
x = Symbol('x')
t = Symbol('t')
s = Symbol('s')

a = 0
b = 1
n = 3

print(Zamena(x,s,a,b,n))
print('==============================')
print(Zamena(x,s,a,b,n).subs(x,'1').evalf())

0.167364032282368*x**3 - 1.0030774044049*x + sin(x) + 1.0030774044049
1.00883501709026


                                Получение ряда Тейлора и Маклорена

In [56]:
def Taylor(f,x,a,n):
    R = []
    for k in range(0,n+1):
        res  = ((1/factorial(k, exact=False))*((x-a)**k)*diff(f, x ,k))
        R.append(res)
    return R

In [84]:
def Maklaren(f,x,a,n):
    R = []
    for k in range(0,n+1):
        res  = ((1/factorial(k, exact=False))*(t**k)*diff(f, x ,k))
        R.append(res)
    return R